In [26]:
import time
from functools import reduce
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import requests

# '노트10 장단점' 검색
url = 'https://www.youtube.com/results?search_query=%EB%85%B8%ED%8A%B810+%EC%9E%A5%EB%8B%A8%EC%A0%90'
raw = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})

html = BeautifulSoup(raw.text,'html.parser')

title_cont = html.select('h3.yt-lockup-title')
link_cont = html.select('div.yt-lockup') # Find tag's name in result of 'html'

titles = [title.select_one('a').attrs['title'] for title in title_cont[1:]]
links = ['https://www.youtube.com' + link.select_one('a').attrs['href'] for link in link_cont[1:]] # [0]: remove ad-link

titles[:10]

['냉정하게! 본 갤럭시노트10 / 10+ 좋은점, 아쉬운점은? - 갤럭시노트10 단점',
 '갤럭시 노트 10+ 3주 실사용 후기! 장단점 3가지 | Galaxy note 10+',
 '실사용자 짬에서 나오는 갤럭시 노트10 or 노트10+ 차이점 13가지! 고민중이라면 꼭! 보세요.',
 '갤럭시노트10 단점만 파보았다. 정말 플러스 모델이 정답일까?!!',
 '갤럭시 노트10이 노트10 플러스보다 더 나은 이유?',
 '좋은거 맞아? 갤럭시노트10  솔직한 느낌 및 달라진점! (갤럭시노트9 차이, 장단점, 성능)',
 '삼성 갤럭시 노트10 리뷰: 작다. 야무지다. [4K]',
 '순삭 6분리뷰 갤럭시노트10 이거보고 결정해도 안늦어! 노트10플러스의 모든것!',
 '[ENG SUB] 역시 아이폰이 짱?! [갤럭시노트10 절대 사지 말아야 하는 이유 7가지]',
 '노트10과 노트10+! 뭘 살지 고민 되신다면 꼭 보세요.']

In [28]:
driver = webdriver.Chrome('Chromedriver.exe')

comments = []
likes = []

# use 10 videos
for link in links[:10]: 
    each_url = link
    driver.get(each_url)
    
    ### [scroll-down]
    scroll_pause_time = 3
    time.sleep(scroll_pause_time)
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(scroll_pause_time)

    for i in range(15): # more than 10 is okay
        body.send_keys(Keys.END)
        time.sleep(scroll_pause_time)
    ### [scroll-down]
    
    ### [get comments and number of likes]
    page = driver.page_source
    html_each = BeautifulSoup(page,'lxml')
    body = html_each.select('div#body')
    
    cmt = [b.select_one('yt-formatted-string#content-text').text for b in body]
    cmtlike = [b.select_one('span#vote-count-middle').text.replace('\n','') for b in body]
    ### [get comments and number of likes]
    
    comments.append(cmt)
    likes.append(cmtlike)

df = pd.DataFrame(
    {'commentText': reduce(lambda x,y : x+y, comments), 
     'likes': reduce(lambda x,y : x+y, likes)},
    columns = ['commentText', 'likes']
    )

df.head()

,commentText,likes
0,플러스로 유도하는 전략이 잘못됐다에 크게 공감합니다 작은폰을 원하는 사람도 똑같은 ...,71
1,애플워치차고 갤럭시 리뷰라...\n(역시 냉정하군ㅋㅋㅋㅋㅋ),68
2,삼성의 갤노트10+몰아주기는 유투버들에게도 전달된건지..\n대부분 플러스를 집중적으...,29
3,크기에 따른 스펙을 구분한다는 점이 정말 아쉽네요,16
4,리더유님이 지적하는 진정한 단점 제작사가 깊이 가슴에 새겨야 할텐데~👍,13


In [29]:
driver.close()

In [30]:
# df.to_csv('galaxynote10_comments.csv', index=False)